In [1]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
try:
    from cnn_finetune import make_model
except:
    !pip install cnn_finetune
    from cnn_finetune import make_model
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import torch.optim as optim
from glob import glob
from tqdm import tqdm
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

In [2]:
torch.cuda.is_available()

True

In [3]:
def show_image(image):
    plt.imshow(image)
    # pause a bit so that plots are updated
    plt.pause(0.01)

In [4]:
def convert_url_to_local_path(url):
    '''
    gets the location of the image in the shared directory so we don't have to redownload
    '''
    return '/home/jovyan/course/ladi/'+'/'.join(url.split('/')[3:])

def labeltonumpy_i(label):
    int_list = [int(x) for x in (label[1:-1].split(' '))]
    return np.array(int_list)

class MLCNN_Dataset(Dataset):

    def __init__(self, csv_file, label_csv, transform = None):
        """
        Args:
            csv_file (string): Path to the csv file with metadata.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.final_metadata = pd.read_csv(csv_file)
        
        # get the path in the shared directory
        self.final_metadata['local_path'] = self.final_metadata['url'].apply(convert_url_to_local_path)
        
        self.final_label = pd.read_csv(label_csv)
        
        self.final_label['label'] = self.final_label['label'].apply(labeltonumpy_i)
        
        self.final_data = pd.merge(self.final_metadata, 
                                        self.final_label,
                                       on="s3_path").drop(['Unnamed: 0_x','Unnamed: 0_y'],axis = 1)
        self.transform = transform
        
    def __len__(self):
        return len(self.final_metadata)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        ## Load images from shared directory. There is no need to download images to local machine. ##
        local_path = self.final_metadata.iloc[idx]['local_path']
        url = self.final_metadata.iloc[idx]['url']
        try:
            image = Image.fromarray(io.imread(local_path))
            img_name = local_path
        except:
            image = Image.fromarray(io.imread(url))
            img_name = url
        uuid = self.final_data.iloc[idx, 1]
        timestamp = self.final_data.iloc[idx, 2]
        gps_lat = self.final_data.iloc[idx, 3]
        gps_lon = self.final_data.iloc[idx, 4]
        gps_alt = self.final_data.iloc[idx, 5]
        file_size = self.final_data.iloc[idx, 6]
        width = self.final_data.iloc[idx, 7]
        height = self.final_data.iloc[idx, 8]
        label = self.final_data.iloc[idx, -1]
        
        if self.transform:
            image = self.transform(image)

        sample = {'image': image, 'label': label}#, 'uuid': uuid, 'timestamp': timestamp, 'gps_lat': gps_lat, 'gps_lon': gps_lon, 'gps_alt': gps_alt, 'orig_file_size': file_size, 'orig_width': width, 'orig_height': height}

        return sample

final_dataset = MLCNN_Dataset(csv_file = 'final_metadata.csv', label_csv = 'changedLabels.csv')

scale = transforms.Resize(768)
crop = transforms.RandomCrop(512)
rotate = transforms.RandomRotation(25)
#flip_demo = transforms.RandomHorizontalFlip(1) # flip with 100% chance just to demo
flip = transforms.RandomHorizontalFlip(p=0.5)
composed = transforms.Compose([scale,
                               crop,
                               rotate,
                               flip,
                                transforms.ToTensor()])

Transformed_dataset = MLCNN_Dataset(csv_file = 'final_metadata.csv', label_csv = 'changedLabels.csv',
                                          transform = transforms.Compose([scale, crop, rotate, flip, transforms.ToTensor()]))

In [5]:
Transformed_dataset.final_data.head()

,uuid,timestamp,gps_lat,gps_lon,gps_alt,file_size,width,height,s3_path,url,local_path,label
0,f3a9a1d1324b747b16d5ddd88b1e582d4430d952,2015-10-08 12:33:11,33.642213,-79.835887,292.0,6388659.0,6000.0,4000.0,s3://ladi/Images/FEMA_CAP/1013/20195/DSC_0512_...,https://ladi.s3-us-west-2.amazonaws.com/Images...,/home/jovyan/course/ladi/Images/FEMA_CAP/1013/...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,489efbec368d78a08e08aef15f21409745231328,2015-10-08 12:33:32,33.647137,-79.829637,306.0,6283783.0,6000.0,4000.0,s3://ladi/Images/FEMA_CAP/1013/20195/DSC_0516_...,https://ladi.s3-us-west-2.amazonaws.com/Images...,/home/jovyan/course/ladi/Images/FEMA_CAP/1013/...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,38b2cdf28c9f06cf3adac2f1dd0c50332e3a3ab9,2015-10-08 12:33:43,33.650238,-79.834105,301.0,6990298.0,6000.0,4000.0,s3://ladi/Images/FEMA_CAP/1013/20195/DSC_0521_...,https://ladi.s3-us-west-2.amazonaws.com/Images...,/home/jovyan/course/ladi/Images/FEMA_CAP/1013/...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,405c090c177e5f22cabc7ff4124dbbdd66bd3413,2015-10-08 12:35:10,33.611918,-79.836295,331.0,6525512.0,6000.0,4000.0,s3://ladi/Images/FEMA_CAP/1013/20195/DSC_0529_...,https://ladi.s3-us-west-2.amazonaws.com/Images...,/home/jovyan/course/ladi/Images/FEMA_CAP/1013/...,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,d3f72158df1bea3fbe99c1b9c0df2c1bc1ab1a94,2015-10-09 11:01:41,33.476848,-79.556262,388.0,6244449.0,6000.0,4000.0,s3://ladi/Images/FEMA_CAP/1013/20211/DSC_1028_...,https://ladi.s3-us-west-2.amazonaws.com/Images...,/home/jovyan/course/ladi/Images/FEMA_CAP/1013/...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [6]:
batch_size = 16
test_split_ratio = .3
shuffle_dataset = True
random_seed= 76

# Creating data indices for training and validation splits:
dataset_size = len(Transformed_dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split_ratio * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

# Creating data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(Transformed_dataset, batch_size=batch_size,
                                           sampler=train_sampler, num_workers = 2)
test_loader = torch.utils.data.DataLoader(Transformed_dataset, batch_size=16,
                                                sampler=test_sampler, num_workers = 0)

In [7]:

#Medium Code
# files = glob('img_align_celeba/*.jpg')
# shuffle = np.random.permutation(len(files))
# for i in ['train', 'valid']:
#     os.mkdir(os.path.join('/tmp/', i))
    
# valid_dict = {}
# valid_file_names = []
# for i in tqdm(shuffle[:60780]):
#     file_name = files[i].split('/')[-1]
#     labels = np.array(label_df[label_df.index==file_name])
#     valid_dict[file_name] = labels
#     valid_file_names.append(file_name)
#     os.rename(files[i], os.path.join('/tmp/', '/tmp/valid', file_name))
# valid_df.index = valid_file_names
# valid_df.columns = ['labels']

## uncomment the below given line to check the head of the dataframe
# valid_df.head()

In [8]:
class MultiClassifier(nn.Module):
    def __init__(self):
        super(MultiClassifier, self).__init__()
        self.ConvLayer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3), # 3, 256, 256
            nn.MaxPool2d(2), # op: 16, 127, 127
            nn.ReLU(), # op: 64, 127, 127
        )
        self.ConvLayer2 = nn.Sequential(
            nn.Conv2d(64, 128, 3), # 64, 127, 127   
            nn.MaxPool2d(2), #op: 128, 63, 63
            nn.ReLU() # op: 128, 63, 63
        )
        self.ConvLayer3 = nn.Sequential(
            nn.Conv2d(128, 256, 3), # 128, 63, 63
            nn.MaxPool2d(2), #op: 256, 30, 30
            nn.ReLU() #op: 256, 30, 30
        )
        self.ConvLayer4 = nn.Sequential(
            nn.Conv2d(256, 512, 3), # 256, 30, 30
            nn.MaxPool2d(2), #op: 512, 14, 14
            nn.ReLU(), #op: 512, 14, 14
            nn.Dropout(0.2)
        )
        self.Linear1 = nn.Linear(460800, 1024)
        self.Linear2 = nn.Linear(1024, 256)
        self.Linear3 = nn.Linear(256, 15)
        
        
    def forward(self, x):
        x = self.ConvLayer1(x)
        x = self.ConvLayer2(x)
        x = self.ConvLayer3(x)
        x = self.ConvLayer4(x)
        x = x.view(x.size(0),-1)
        x = self.Linear1(x)
        x = self.Linear2(x)
        x = self.Linear3(x)
        return F.sigmoid(x)

In [9]:
model = MultiClassifier().cuda()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0006, momentum = 0.9, nesterov=True)

In [10]:
model = make_model('alexnet', num_classes=15, pretrained=True, input_size = (768,768)).cuda()
#model = make_model('resnet18', num_classes=2, pretrained=True).cuda()

In [11]:
import torchvision.models as models
from torchvision.models.resnet import ResNet, BasicBlock

class MyCustomResnet18(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        
        resnet18 = models.resnet18(pretrained=pretrained)
        # here we get all the modules(layers) before the fc layer at the end
        # note that currently at pytorch 1.0 the named_children() is not supported
        # and using that instead of children() will fail with an error
        self.features = nn.ModuleList(resnet18.children())[:-1]
        # Now we have our layers up to the fc layer, but we are not finished yet 
        # we need to feed these to nn.Sequential() as well, this is needed because,
        # nn.ModuleList doesnt implement forward() 
        # so you cant do sth like self.features(images). Therefore we use 
        # nn.Sequential and since sequential doesnt accept lists, we 
        # unpack all the items and send them like this
        self.features = nn.Sequential(*self.features)
        # now lets add our new layers 
        in_features = resnet18.fc.in_features
        # from now, you can add any kind of layers in any quantity!  
        # Here I'm creating three new layers 
        self.Linear1 = nn.Linear(512, 1024)
        self.Linear2 = nn.Linear(1024, 256)
        self.Linear3 = nn.Linear(256, 15)
        
        # initialize all fc layers to xavier
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_normal_(m.weight, gain = 1)

    def forward(self, input_imgs):
       # now in forward pass, you have the full control, 
       # we can use the feature part from our pretrained model  like this
        output = self.features(input_imgs)
        # since we are using fc layers from now on, we need to flatten the output.
        # we used the avgpooling but we still need to flatten from the shape (batch, 1,1, features)
        # to (batch, features) so we reshape like this. input_imgs.size(0) gives the batchsize, and 
        # we use -1 for inferring the rest
        x = output.view(input_imgs.size(0), -1)
       # and also our new layers. 
        x = self.Linear1(x)
        x = self.Linear2(x)
        x = self.Linear3(x)
        return F.sigmoid(x)


model = MyCustomResnet18().cuda()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0006, momentum = 0.9, nesterov=True)
# if you need pretrained weights

In [ ]:
#Opens saved model
#model.load_state_dict(torch.load(path + network_name))

In [12]:
def pred_acc(original, predicted):
    predicted = torch.round(predicted)
    for i in range(len(predicted)):
        if predicted[i] == 0:
            predicted[i] = 2
    return predicted.eq(original).sum().numpy()/sum(original)

In [13]:
#our code
torch.autograd.set_detect_anomaly = False
torch.backends.cudnn.benchmark = True
def fit_model(epochs, model, dataloader, phase = 'training', volatile = False):
    
    pprint("Epoch: {}".format(epochs))
    
    if phase == 'training':
        model.train()
        
    if phase == 'validation':
        model.eval()
        volatile = True
        
    running_loss = []
    running_acc = []
    b = 0
    #print('a')
    for i, data in tqdm(enumerate(dataloader)):
        #print('b')

        inputs, target = data['image'].cuda(), data['label'].float().cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
            #print('c')
        ops = model(inputs)
        
        acc_ = []
        for j, d in enumerate(ops, 0):
            #print('d')
            acc = pred_acc(torch.Tensor.cpu(target[j]), torch.Tensor.cpu(d))
            acc_.append(acc)

        loss = criterion(ops, target)
                
        running_loss.append(loss.item())
        full_acc = np.asarray(acc_).mean()
        running_acc.append(full_acc)
        b += 1
        #print('e')
        if phase == 'training':
            
            loss.backward()
        
            optimizer.step()
            #print('f')
        print(f'[epoch {epochs}, batch {i +1} ] average loss: {loss.item()}, average acc: {full_acc}')
            
    total_batch_loss = np.asarray(running_loss).mean()
    total_batch_acc = np.asarray(running_acc).mean()
    #print('g')

    pprint("{} loss is {} ".format(phase,total_batch_loss))
    pprint("{} accuracy is {} ".format(phase, total_batch_acc))
    
    return total_batch_loss, total_batch_acc

In [14]:
!nvidia-smi
#Make sure GPU doesn't forget to be a GPU

Thu Jul 30 14:35:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   56C    P0    38W /  70W |   4707MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [15]:
#This better work

# --> Ctrl + Enter at your own risk <--

trn_losses = []; trn_acc = []
val_losses = []; val_acc = []
for i in tqdm(range(1, 10)):
    trn_l, trn_a = fit_model(i, model, train_loader)
    val_l, val_a = fit_model(i, model, test_loader, phase = 'validation')
    trn_losses.append(trn_l); trn_acc.append(trn_a)
    val_losses.append(val_l); val_acc.append(val_a)

  0%|          | 0/9 [00:00<?, ?it/s]

'Epoch: 1'



0it [00:00, ?it/s]
1it [00:12, 12.35s/it]

[epoch 1, batch 1 ] average loss: 0.8171446919441223, average acc: 0.5937500018626451



2it [00:12,  8.71s/it]

...

[epoch 2, batch 87 ] average loss: 0.14269419014453888, average acc: 0.65625



88it [06:39,  3.81s/it]

[epoch 2, batch 88 ] average loss: 0.15375776588916779, average acc: 0.59375


88it [06:41,  4.56s/it]
 11%|█         | 1/9 [3:09:03<25:12:30, 11343.85s/it]


KeyboardInterrupt: 

In [16]:
network_name = 'SGD_lr0.0006_epoch2start_Resnet18'
torch.save(model, '{}.pth'.format(network_name))

In [17]:
import pickle
with open("{}_trnloss.txt".format(network_name), "wb") as fp:   #Pickling
    pickle.dump(trn_losses, fp) 
with open("{}_trnacc.txt".format(network_name), "wb") as fp:    #Pickling
    pickle.dump(trn_acc, fp) 
with open("{}_valloss.txt".format(network_name), "wb") as fp:   #Pickling
    pickle.dump(val_losses, fp) 
with open("{}_valacc.txt".format(network_name), "wb") as fp:    #Pickling
    pickle.dump(val_acc, fp)

In [ ]:
tor